In [7]:
import collections
from collections import defaultdict
import sys
import json
import random
from jsmin import jsmin
from io import StringIO
import numpy as np
import copy
import importlib
from functools import partial
import math
import os
import compress_pickle
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.insert(0, '/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc')

# from tools_pattern import get_eucledean_dist

script_n = 'distribution_123share_bouton_convergence_210512_random'

import my_plot
importlib.reload(my_plot)
from my_plot import MyPlotData, my_box_plot

def to_ng_coord(coord):
    return (
        int(coord[0]/4),
        int(coord[1]/4),
        int(coord[2]/40),
        )

import compress_pickle

fname = ('/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc/' \
#                                    'mf_grc_model/input_graph_210407_all.gz')
         'gen_db/mf_grc/input_graph_210520_all_100_2.gz')

input_graph = compress_pickle.load(fname)
# GrCs are fully reconstructed and proofread from 90k to 150k

# 800+
z_min = 19800-5000
z_max = 29800+5000
x_min = 320*1000
x_max = 560*1000

# 400+
z_min = 19800
z_max = 29800
x_min = 320*1000
x_max = 560*1000

#270
z_min = 19800
z_max = 29800
x_min = (320+40)*1000
x_max = (560-40)*1000

#550
z_min = 19800-5000
z_max = 29800+5000
x_min = (320+40)*1000
x_max = (560-40)*1000

# #550
# z_min = 19800-5000
# z_max = 29800+5000
# x_min = (320+80)*1000
# x_max = (560-80)*1000

# n_randoms = 5
# replication_hist2 = defaultdict(int)
grc_ids = set()
# mf_ids = set()
# replicated_2shares = defaultdict(int)

def get_prob(in_graph):
    unique_count = True
    count_within_box = True
    overshare_pairs = defaultdict(int)
    processed = set()
    for grc_i_id in in_graph.grcs:
        grc_i = in_graph.grcs[grc_i_id]
        x, y, z = grc_i.soma_loc
        if count_within_box:
            if x < x_min or x > x_max:
                continue
            if z < z_min or z > z_max:
                continue
        grc_ids.add(grc_i.soma_loc)
        rosettes_i = set([mf[1] for mf in grc_i.edges])
#         print(grc_i.edges);asdf
        for grc_j_id in in_graph.grcs:
            if grc_i_id == grc_j_id:
                continue
            if unique_count and (grc_i_id, grc_j_id) in processed:
                continue
            processed.add((grc_i_id, grc_j_id))
            processed.add((grc_j_id, grc_i_id))
            grc_j = in_graph.grcs[grc_j_id]
            x, y, z = grc_j.soma_loc
            if count_within_box:
                if x < x_min or x > x_max:
                    continue
                if z < z_min or z > z_max:
                    continue
            common_rosettes = set([mf[1] for mf in grc_j.edges])
            common_rosettes = common_rosettes & rosettes_i
            pair = tuple(sorted((grc_i.soma_loc, grc_j.soma_loc)))
            overshare_pairs[pair] = len(common_rosettes)
    return overshare_pairs


In [10]:
n_random = 100
rounds = []
for n in range(n_random):
    print('', end='.')
    input_observed = input_graph
    # input_observed = copy.deepcopy(input_graph)
    # input_observed.remove_dendrites(min_len=max_dendrite, max_len=None)
    input_observed.randomize_graph_by_grc2(
#         constant_dendrite_length=22000,
        mf_dist_margin=10000,
        seed=n,
    )
    data = get_prob(input_observed)
    rounds.append(data)

....................................................................................................

In [11]:
import compress_pickle
fname = f'{script_n}_dist_{x_min}_{x_max}_{z_min}_{z_max}_{n_random}.gz'
print(fname)
compress_pickle.dump(rounds, fname)

distribution_123share_bouton_convergence_210512_random_dist_360000_520000_14800_34800_100.gz
